In [1]:
import pandas as pd
from evaluate import *
from svd import *
from bert import *
from document import *
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jensenshannon
%load_ext autoreload
%autoreload 2

In [2]:
tfidf_fname = "assets/tfidf"

In [3]:
data = pd.read_csv("metadata.csv")
topics = load_topics()
queries = get_queries_from_topics(topics)

/home/selo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
trec_ir = BertRanker("assets/tfidf")

In [12]:
corpus=data

In [20]:
corpus["cord_uid"]

0         ug7v899j
1         02tnwd4m
2         ejv2xln0
3         2b73a28n
4         9785vg6d
            ...   
192504    z4ro6lmh
192505    hi8k8wvb
192506    ma3ndg41
192507    wh10285j
192508    pnl9th2c
Name: cord_uid, Length: 192509, dtype: object

In [16]:
corpus["cord_uid"]


0         ug7v899j
1         02tnwd4m
2         ejv2xln0
3         2b73a28n
4         9785vg6d
            ...   
192504    z4ro6lmh
192505    hi8k8wvb
192506    ma3ndg41
192507    wh10285j
192508    pnl9th2c
Name: cord_uid, Length: 192509, dtype: object

In [35]:
PUNCS = "!\"#$%&()*+,-./:;<=>?@[\\]^_{|}~"
PUNCS_RE = re.compile("([{}])".format(PUNCS), re.IGNORECASE | re.DOTALL)
STOPWORDS_SET = set(stopwords.words('english'))
THRESHOLD_MIN_TOKEN = 1
def process_text( text):
    text = text.replace("-\n", " ")
    text = PUNCS_RE.sub(" ", text)
    text = text.lower()
    tokens = [word for word in word_tokenize(text) 
                  if (word.isalpha() and len(word) > 1)]
    tokens = [w for w in tokens if not w in STOPWORDS_SET]
    tokens = [trec_ir.porter.stem(t) for t in tokens]
    return tokens

In [39]:
dir(corpus)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 

In [44]:
q = corpus.iloc[1]

In [45]:
q["title"]

'Nitric oxide: a pro-inflammatory mediator in lung disease?'

In [26]:
cord_set = set(trec_ir.doc_ids)
corpus_d = {}
for i in range(len(corpus)):
    if (corpus["cord_uid"][i] in cord_set) and (corpus["cord_uid"][i] not in corpus_d):
        corpus_d[corpus["cord_uid"][i]] = corpus.iloc[i]


In [29]:
N = len(trec_ir.doc_ids)

In [36]:
doc_texts = []
for cord_uid in trec_ir.doc_ids:
    d = corpus_d[cord_uid]
    title = corpus["title"][i]
    title = "" if (not isinstance(title, str)) else title
    abstract = corpus["abstract"][i]
    abstract = "" if (not isinstance(abstract, str)) else abstract
    text= title + " " + abstract
    text = " ".join(process_text(text))
    doc_texts.append(text)

In [37]:
embeddings = trec_ir.model.encode(doc_texts, show_progress_bar=True)

KeyboardInterrupt: 